MODEL CNN

In [1]:
import keras
import helpers
# import brewer2mpl
import numpy as np
import matplotlib.pyplot as plt 

# set3 = brewer2mpl.get_map('Set3', 'qualitative', 4).mpl_colors
%matplotlib inline

from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.models import Sequential

import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib
from keras.models import model_from_json
from keras.optimizers import SGD
from tensorflow.python.framework import graph_util

from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report

C:\Users\COEGIN\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 64
num_classes = 36
epochs = 50
img_rows, img_cols = 28, 28

print('Start loading data...\n')
files, labels = helpers.load_chars74k_data()
X, y = helpers.create_dataset(files, labels)
print('Data has been loaded')

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=2, train_size=0.8)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train/=255.
x_test/=255.

print('Data latih :', x_train.shape[0])
print('Data Uji : ', x_test.shape[0],'\n')

print('Total Data : ', x_train.shape[0]+ x_test.shape[0])

Start loading data...

Data has been loaded


C:\Users\COEGIN\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Data latih : 56581
Data Uji :  14146 

Total Data :  70727


In [3]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [4]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# train_generator, validation_generator = helpers.create_datagenerator(x_train, x_test, y_train, y_test)

# Convolutional network will be build with Keras.
print('Start training the model.')
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=3, strides=1, \
        padding='same', activation='relu', \
        input_shape=[28, 28, 1]))
# 28*28*64
model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
# 14*14*64

model.add(Conv2D(filters=128, kernel_size=3, strides=1, \
        padding='same', activation='relu'))
# 14*14*128
model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
# 7*7*128

model.add(Conv2D(filters=256, kernel_size=3, strides=1, \
        padding='same', activation='relu'))
# 7*7*256
model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
# 4*4*256

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(36, activation='softmax'))

sgd = SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=sgd,
              metrics=['accuracy'])

'''
saves the model weights after each epoch if the validation loss decreased
'''
checkpointer = ModelCheckpoint(filepath='model/modelchars28.h5', verbose=2, save_best_only=True)

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test), callbacks=[checkpointer])

Start training the model.
Train on 56581 samples, validate on 14146 samples
Epoch 1/50
56581/56581 [==============================] - 545s 10ms/step - loss: 1.0442 - acc: 0.7137 - val_loss: 0.3248 - val_acc: 0.9090

Epoch 00001: val_loss improved from inf to 0.32480, saving model to model/modelchars28.h5
Epoch 2/50
56581/56581 [==============================] - 572s 10ms/step - loss: 0.3309 - acc: 0.9046 - val_loss: 0.2386 - val_acc: 0.9303

Epoch 00002: val_loss improved from 0.32480 to 0.23862, saving model to model/modelchars28.h5
Epoch 3/50
56581/56581 [==============================] - 563s 10ms/step - loss: 0.2360 - acc: 0.9304 - val_loss: 0.1818 - val_acc: 0.9462

Epoch 00003: val_loss improved from 0.23862 to 0.18178, saving model to model/modelchars28.h5
Epoch 4/50
56581/56581 [==============================] - 575s 10ms/step - loss: 0.1753 - acc: 0.9481 - val_loss: 0.1768 - val_acc: 0.9468

Epoch 00004: val_loss improved from 0.18178 to 0.17685, saving model to model/modelcha

KeyboardInterrupt: 

In [ ]:
print(history.history.keys())  

plt.figure(1)  

# summarize history for accuracy  

plt.subplot(211)  
plt.plot(history.history['acc'])  
plt.plot(history.history['val_acc'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  

# summarize history for loss  


In [ ]:
# print("SGD")
 
plt.subplot(212)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
plt.show()  

# Calculate loss and accuracy.
score = model.evaluate(x_test, y_test, verbose=0)
print('Model has been trained.')
print('Test loss:', score[0])
print('Test accuracy: ', score[1]*100.0)

In [ ]:
# def plot_filters(layer, x, y):
#     filters = layer.W.get_value()
#     fig = plt.figure(figsize=(8, 4))
#     for j in range(len(filters)):
#         ax = fig.add_subplot(y, x, j+1)
#         ax.matshow(filters[j][0], cmap = matplotlib.cm.Blues)
#         plt.xticks(np.array([]))
#         plt.yticks(np.array([]))
#     plt.tight_layout()
#     return plt

# plot_filters(model.layers[0], 8, 4)

In [ ]:
for i, layer in enumerate(model.layers):
    print ("Layer", i, "\t", layer.name, "\t\t", layer.input_shape, "\t\t\t", layer.output_shape)
    

In [ ]:
print(model.summary())

In [ ]:
#Save the model
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5

namaModel = "model.h5"
model.save_weights(namaModel)
print("Model : " + namaModel + " telah disimpan")

## Data Uji

In [ ]:
y_prob = model.predict(x_test, batch_size=batch_size, verbose=0)
y_pred = [np.argmax(prob) for prob in y_prob]
y_true = [np.argmax(true) for true in y_test]

print('jumlah data uji :', len(y_prob))

cm = confusion_matrix(y_true, y_pred)
print(cm)

plt.matshow(cm)
plt.colorbar()

In [ ]:
labels = ['-----0-----', '-----1-----', '-----2-----', '-----3-----', '-----4-----', '-----5-----', '-----6-----', '-----7-----',
         '-----8-----', '-----9-----', '-----a-----', '-----b-----', '-----c-----', '-----d-----', '-----e-----', '-----f-----',
         '-----g-----', '-----h-----', '-----i-----', '-----j-----', '-----k-----', '-----l-----', '-----m-----', '-----n-----', 
         '-----o-----', '-----p-----', '-----q-----', '-----r-----', '-----s-----', '-----t-----', '-----u-----', '-----v-----', 
         '-----w-----', '-----x-----', '-----y-----', '-----z-----']

def class_precision(y_true, y_pred, character):
    cm = confusion_matrix(y_true, y_pred)
    i = [i for i, label in enumerate(labels) if label == character][0]
    col = [cm[j,i] for j in range(0,36)]
    return float(col[i])/sum(col)

def class_recall(y_true, y_pred, character):
    cm = confusion_matrix(y_true, y_pred)
    i = [i for i, label in enumerate(labels) if label == character][0]
    row = [cm[i,j] for j in range(0,36)]
    return float(row[i])/sum(row)

def class_accuracy(y_true, y_pred, character):
    cm = confusion_matrix(y_true, y_pred)
    i = [i for i, label in enumerate(labels) if label == character][0]
    tp = cm[i,i]
    fn = sum([cm[i,j] for j in range(0,36) if j != i])
    fp = sum([cm[j,i] for j in range(0,36) if j != i])
    tn = sum([cm[i,j] for j in range(0,36) for i in range(0,4)]) -(tp+fp+fn)
    return float(tp + tn)/sum([tp, fn, fp, tn])

In [ ]:
for character in labels:
    print(character.upper())
    print('   acc = {0:.2f}'.format(class_accuracy(y_true, y_pred, character)))
    print('  prec = {0:.2f}'.format(class_precision(y_true, y_pred, character)))
    print('recall = {0:.2f}\n'.format(class_recall(y_true, y_pred, character)))
    
print('===============================================================')
print(classification_report(y_true, y_pred, target_names=labels))

## Data Latih

In [ ]:
y_prob = model.predict(x_train, batch_size=batch_size, verbose=0)
y_pred = [np.argmax(prob) for prob in y_prob]
y_true = [np.argmax(true) for true in y_train]

print('jumlah data latih :', len(y_prob))

cm = confusion_matrix(y_true, y_pred)
print(cm)

plt.matshow(cm)
plt.colorbar()

In [ ]:
for character in labels:
    print(character.upper())
    print('   acc = {0:.2f}'.format(class_accuracy(y_true, y_pred, character)))
    print('  prec = {0:.2f}'.format(class_precision(y_true, y_pred, character)))
    print('recall = {0:.2f}\n'.format(class_recall(y_true, y_pred, character)))
    
print('===============================================================')
print(classification_report(y_true, y_pred, target_names=labels))

In [ ]:
# K.set_learning_phase(0)
# sess = K.get_session()

# output_node_name = "dense_2/Softmax" # Name of your output node

# with sess as sess:
#     init_op = tf.global_variables_initializer()
#     sess.run(init_op)
#     graph_def = sess.graph.as_graph_def()
#     output_graph_def = graph_util.convert_variables_to_constants(
#                                                                  sess,
#                                                                  sess.graph.as_graph_def(),
#                                                                  output_node_name.split(","))
#     tf.train.write_graph(output_graph_def,
#                          logdir="model",
#                          name="my_model.pb",
#                          as_text=False)